# Topic Modelling and Text Categorisation

In [ ]:
import csv

import pandas as pd

import utils

from datasets import load_dataset, Dataset

In [30]:
dataset = load_dataset("AmazonScience/mintaka", "all", split="train")

In [33]:
for i in range(0, 50):
    print(dataset[i])

{'id': 'a9011ddf', 'lang': 'en', 'question': 'What is the seventh tallest mountain in North America?', 'answerText': 'Mount Lucania', 'category': 'geography', 'complexityType': 'ordinal', 'questionEntity': [{'name': 'Q49', 'entityType': 'entity', 'label': 'North America', 'mention': 'North America', 'span': [40, 53]}, {'name': '7', 'entityType': 'ordinal', 'label': '', 'mention': 'seventh', 'span': [12, 19]}], 'answerEntity': [{'name': 'Q1153188', 'label': 'Mount Lucania'}]}
{'id': '2723bb1b', 'lang': 'en', 'question': 'Which actor was the star of Titanic and was born in Los Angeles, California?', 'answerText': 'Leonardo DiCaprio', 'category': 'movies', 'complexityType': 'intersection', 'questionEntity': [{'name': 'Q44578', 'entityType': 'entity', 'label': 'Titanic', 'mention': 'Titanic', 'span': [28, 35]}, {'name': 'Q65', 'entityType': 'entity', 'label': 'Los Angeles', 'mention': 'Los Angeles, California', 'span': [52, 75]}], 'answerEntity': [{'name': 'Q38111', 'label': 'Leonardo DiCa

In [25]:
print(dataset)
categories = set()
english_langs = 0
for entry in dataset:
    categories.add(entry["category"])
    if entry["lang"] == "en":
        english_langs += 1
    

Dataset({
    features: ['id', 'lang', 'question', 'answerText', 'category', 'complexityType', 'questionEntity', 'answerEntity'],
    num_rows: 126000
})


In [23]:
categories

{'books',
 'geography',
 'history',
 'movies',
 'music',
 'politics',
 'sports',
 'videogames'}

In [26]:
english_langs

14000

In [36]:
file_path = r"C:\Users\jayde\OneDrive\School\Text Mining for AI\project_text_mining\final_project_tm\data\train_data\topic-train.tsv"

Call the following to convert the data to a TSV file:

```python
utils.topic_data_to_file(raw_data, file_name)
```

In [41]:
df = pd.read_csv(file_path, sep="\t")

In [43]:
df.head()

,id,question,category
0,a9011ddf,What is the seventh tallest mountain in North ...,geography
1,2723bb1b,Which actor was the star of Titanic and was bo...,movie
2,88349c89,Which actor starred in Vanilla Sky and was mar...,movie
3,bff78c91,What year was the first book of the A Song of ...,book
4,982450cf,Who is the youngest current US governor?,politics


## Training the Model